In [1]:
# ignore warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report
import eli5

In [3]:
data = pd.read_csv("data/ner_dataset.csv", encoding="utf-8")
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,Tag
369382,Sentence: 16858,vùng,O
369383,Sentence: 16858,chôn_cất,O
369384,Sentence: 16858,người_thân,O
369385,Sentence: 16858,khi,O
369386,Sentence: 16858,qua_đời,O
369387,Sentence: 16858,",",O
369388,Sentence: 16858,gọi,O
369389,Sentence: 16858,là,O
369390,Sentence: 16858,nhị_tì,O
369391,Sentence: 16858,...,O


In [4]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(), s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [5]:
getter = SentenceGetter(data)
sent = getter.get_next()
print(sent)

[('Đó', 'O'), ('là', 'O'), ('con', 'O'), ('đường', 'O'), ('biển', 'O'), ('ngắn', 'O'), ('nhất', 'O'), ('để', 'O'), ('đi', 'O'), ('từ', 'O'), ('Ấn_Độ_Dương', 'LOCATION'), ('sang', 'O'), ('Thái_Bình_Dương', 'LOCATION'), (',', 'O'), ('chiếm', 'O'), ('đến', 'O'), ('lượng', 'O'), ('hàng_hoá', 'O'), ('lưu_thông', 'O'), ('đường_biển', 'O'), ('của', 'O'), ('thế_giới', 'O'), (',', 'O'), ('đó', 'O'), ('là', 'O'), ('hải_trình', 'O'), ('lớn', 'O'), ('nhất', 'O'), ('từ', 'O'), ('tây', 'O'), ('sang', 'O'), ('đông', 'O'), ('với', 'O'), ('50.000', 'O'), ('lượt', 'O'), ('tàu_bè', 'O'), ('qua_lại', 'O'), ('mỗi', 'O'), ('năm', 'O'), ('...', 'O')]


In [6]:
sentences = getter.sentences

In [7]:
def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [9]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
print(X[0]) # list of dictionaries of features
print(y[0]) # list of tags

[{'BOS': True, 'word.lower()': 'đó', 'word.istitle()': True, 'bias': 1.0, 'word[-3:]': 'Đó', 'word[-2:]': 'Đó', '+1:word.istitle()': False, 'word.isupper()': False, '+1:word.lower()': 'là', '+1:word.isupper()': False, 'word.isdigit()': False}, {'-1:word.istitle()': True, '-1:word.lower()': 'đó', 'word[-2:]': 'là', '-1:word.isupper()': False, '+1:word.lower()': 'con', 'word.lower()': 'là', 'word.istitle()': False, 'bias': 1.0, 'word[-3:]': 'là', '+1:word.istitle()': False, 'word.isupper()': False, '+1:word.isupper()': False, 'word.isdigit()': False}, {'-1:word.istitle()': False, '-1:word.lower()': 'là', 'word[-2:]': 'on', '-1:word.isupper()': False, '+1:word.lower()': 'đường', 'word.lower()': 'con', 'word.istitle()': False, 'bias': 1.0, 'word[-3:]': 'con', '+1:word.istitle()': False, 'word.isupper()': False, '+1:word.isupper()': False, 'word.isdigit()': False}, {'-1:word.istitle()': False, '-1:word.lower()': 'con', 'word[-2:]': 'ng', '-1:word.isupper()': False, '+1:word.lower()': 'biển'


['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOCATION', 'O', 'LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [12]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [13]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

    LOCATION       0.85      0.85      0.85      9028
        MISC       0.91      0.92      0.91       561
           O       0.99      1.00      0.99    345533
ORGANIZATION       0.76      0.59      0.66      3268
      PERSON       0.93      0.85      0.89     11002

 avg / total       0.98      0.99      0.98    369392



In [14]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [17]:
eli5.show_weights(crf, top=30)

In [10]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [11]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [12]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

    LOCATION       0.78      0.73      0.75      9028
        MISC       0.67      0.75      0.71       561
           O       0.99      1.00      0.99    345533
ORGANIZATION       0.63      0.37      0.47      3268
      PERSON       0.82      0.80      0.81     11002

 avg / total       0.98      0.98      0.98    369392



In [13]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=10, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [14]:
eli5.show_weights(crf, top=30)